In [96]:
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2
import numpy as np
import random
import os
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pandas as pd




In [ ]:
FER2013 = load_model('Weights/FER2013_BestYet.keras')
AffectNET = load_model('Weights/AffectNet_Final.keras')
SER = load_model('Weights/best_weights.keras')
tokenizer = AutoTokenizer.from_pretrained("michellejieli/emotion_text_classifier")
textModel = AutoModelForSequenceClassification.from_pretrained("michellejieli/emotion_text_classifier")
textModel.load_state_dict(torch.load("Weights/trained_emotion_text_classifier_3.pth"))
emotionMapping = {'0': 'Anger','1': 'Fear','2': 'Happiness', '3': 'Sadness', '4': 'Surprise', '5': 'Neutral'}
Textemotion_labels = ["Sadness", "Happiness", "Neutral", "Anger", "Fear", "Surprise"]



/tmp/ipykernel_2285046/477453432.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  textModel.load_state_dict(torch.load("Weights/trained_emotion_text_classifier_3.pth"))


In [98]:
print('SER input shape: ',SER.input_shape)
print('FER2013 input shape: ',FER2013.input_shape)
print('AffectNET input shape: ',AffectNET.input_shape)
textModel.eval()

SER input shape:  (None, 335, 50)
FER2013 input shape:  (None, 48, 48, 1)
AffectNET input shape:  (None, 96, 96, 3)


RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-5): 6 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
           

In [142]:
reverse_mapping = {v: k for k, v in emotionMapping.items()}

TextClasifiedLabels = {}
for i, label in enumerate(Textemotion_labels):
    if label in reverse_mapping:
        TextClasifiedLabels[i] = int(reverse_mapping[label])  # map old index to new index

print("Mapped label indices:", TextClasifiedLabels)

Mapped label indices: {0: 3, 1: 2, 2: 5, 3: 0, 4: 1, 5: 4}


In [143]:
def predict_emotion(model, text):

    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
        probs = torch.nn.functional.softmax(logits, dim=1)
        predicted_class = torch.argmax(probs, dim=1).item()

    predicted_emotion = Textemotion_labels[predicted_class]
    return predicted_emotion,predicted_class, probs.numpy()


In [119]:
def fer2013inputshape(img):
    img = cv2.resize(img, (48, 48))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = np.expand_dims(img, axis=-1)
    img = np.expand_dims(img, axis=0)
    return img

In [120]:
def AffectNetinputshape(img):
    img = cv2.resize(img, (96, 96))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = np.expand_dims(img, axis=-1)
    img = np.expand_dims(img, axis=0)
    return img

In [121]:
#SER input shape:  (None, 335, 50)
def SERinputshape(wav):
    #wav = wav[0:335]
    wav = np.pad(wav, (0, 335-len(wav)), 'constant', constant_values=(0, 0))
    wav = np.expand_dims(wav, axis=-1)
    wav = np.expand_dims(wav, axis=0)
    return wav

    

In [122]:
img = cv2.imread("/home/group02-f24/Desktop/SadBaby.jpg")
img = img.astype('float32') / 255.0  # Normalize if needed

In [123]:
FER2013img = fer2013inputshape(img)
AffectNETimg = AffectNetinputshape(img)

### RAW PREDICTIONS

In [158]:
FER2013pred = FER2013.predict(FER2013img)
AffectNETpred = AffectNET.predict(AffectNETimg)
emotion, predclass,probs = predict_emotion(textModel,"I am so devastated!")

AffectNETpred = 1.1*AffectNETpred
FER2013pred = 0.9*FER2013pred
probs = 0.8 * probs[0, :6]

reordered_preds = np.zeros_like(probs)

for old_idx, new_idx in TextClasifiedLabels.items():
    reordered_preds[new_idx] = probs[old_idx]

probs = reordered_preds
concPred = FER2013pred + AffectNETpred + probs

print('FER2013 prediction: ',FER2013pred)
print('AffectNET prediction: ',AffectNETpred)
print('Text prediction: ',probs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
FER2013 prediction:  [[0.44925573 0.09617763 0.18659915 0.01830562 0.0162384  0.13342343]]
AffectNET prediction:  [[0.03346886 0.17315489 0.01385948 0.62941283 0.12246295 0.12764111]]
Text prediction:  [4.0775325e-04 8.8411989e-04 5.5360800e-04 7.9721123e-01 6.8269053e-04
 2.6062629e-04]


### Normalized PREDICTIONS

In [159]:
FER2013pred = np.argmax(FER2013pred, axis=1)
AffectNETpred = np.argmax(AffectNETpred, axis=1)
print('Text Classified Emotion',emotionMapping[str(TextClasifiedLabels[predclass])])
print('FER2013 prediction: ',emotionMapping[str(FER2013pred[0])])
print('AffectNET prediction: ',emotionMapping[str(AffectNETpred[0])])

Text Classified Emotion Sadness
FER2013 prediction:  Anger
AffectNET prediction:  Sadness


### Concatenated Predictions

In [160]:
finalPrediction = np.argmax(concPred, axis=1)
print(finalPrediction)
print('FER2013 prediction: ',emotionMapping[str(finalPrediction[0])])


[3]
FER2013 prediction:  Sadness
